# gsf

**gsf** is a library for the processing of geological data. It is composed by two modules: gsf_py, implemented in Python 3, and gsf_hs, that is the corresponding, in-progress version in Haskell.

In this notebook the Python version will be documented.

How to use **gsf_py** after its installation?

**gsf_py** can be imported in Python 3 via the command: 

In [1]:
import gsf_py

We can be the version by issuing:

In [2]:
gsf_py.version

'1.0.1'